# Introduction to Docker

**Learning Objectives**
  * Build and run Docker containers
  * Pull Docker images from Docker Hub and Google Container Registry
  * Push Docker images to Google Container Registry

## Overview

Docker is an open platform for developing, shipping, and running applications. With Docker, you can separate your applications from your infrastructure and treat your infrastructure like a managed application. Docker helps you ship code faster, test faster, deploy faster, and shorten the cycle between writing code and running code.

Docker does this by combining kernel containerization features with workflows and tooling that helps you manage and deploy your applications.

Docker containers can be directly used in Kubernetes, which allows them to be run in the Kubernetes Engine with ease. After learning the essentials of Docker, you will have the skillset to start developing Kubernetes and containerized applications.

## Basic Docker commands

See what docker images you have. 

In [61]:
!docker images

REPOSITORY                                                              TAG       IMAGE ID       CREATED        SIZE
gcr.io/qwiklabs-gcp-04-569268d5f324/tfx_covertype_continuous_training   latest    036086844ebc   12 days ago    4.98GB
gcr.io/qwiklabs-gcp-04-569268d5f324/tfx_covertype_continuous_training   <none>    9dd487f85433   12 days ago    4.98GB
gcr.io/qwiklabs-gcp-04-569268d5f324/node-app                            0.2       fe381b8df6e9   2 weeks ago    884MB
<none>                                                                  <none>    895bef7b3195   2 weeks ago    884MB
<none>                                                                  <none>    5b24b247208c   2 weeks ago    884MB
tensorflow/serving                                                      latest    e874bf5e4700   5 weeks ago    406MB
gcr.io/inverting-proxy/agent                                            <none>    fe507176d0e6   3 months ago   1.73GB
tensorflow/tfx                                        

If this is the first time working with docker you won't have any repositories listed. 

**Note**. If you are running this in an AI Notebook, then you should see a single image `gcr.io/inverting-proxy/agent`. This is the container that is currently running the AI Notebook. 

Let's use `docker run` to pull a docker image called `hello-world` from the public registry. The docker daemon will search for the `hello-world` image, if it doesn't find the image locally, it pulls the image from a public registry called Docker Hub, creates a container from that image, and runs the container for you.

In [62]:
!docker run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world

de127a29: Already exists Digest: sha256:9f6ad537c5132bcce57f7a0a20e317228d382c3cd61edae14650eec68b2b345c
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples

Now when we look at our docker images we should see `hello-world` there as well.

In [63]:
!docker images

REPOSITORY                                                              TAG       IMAGE ID       CREATED        SIZE
gcr.io/qwiklabs-gcp-04-569268d5f324/tfx_covertype_continuous_training   latest    036086844ebc   12 days ago    4.98GB
gcr.io/qwiklabs-gcp-04-569268d5f324/tfx_covertype_continuous_training   <none>    9dd487f85433   12 days ago    4.98GB
gcr.io/qwiklabs-gcp-04-569268d5f324/node-app                            0.2       fe381b8df6e9   2 weeks ago    884MB
<none>                                                                  <none>    895bef7b3195   2 weeks ago    884MB
<none>                                                                  <none>    5b24b247208c   2 weeks ago    884MB
tensorflow/serving                                                      latest    e874bf5e4700   5 weeks ago    406MB
hello-world                                                             latest    d1165f221234   3 months ago   13.3kB
gcr.io/inverting-proxy/agent                          

This is the image pulled from the Docker Hub public registry. The Image ID is in `SHA256` hash format—this field specifies the Docker image that's been provisioned. When the docker daemon can't find an image locally, it will by default search the public registry for the image. Let's run the container again:

Now, if we want to run `docker run hello-world` again, it won't have to download from the container registry.

To see all docker containers running, use `docker ps`.

In [64]:
!docker ps

CONTAINER ID   IMAGE                                              COMMAND                  CREATED       STATUS       PORTS                  NAMES
d5fea7d88c37   gcr.io/qwiklabs-gcp-04-569268d5f324/node-app:0.2   "node ./src/app.js"      2 weeks ago   Up 2 weeks   0.0.0.0:4000->80/tcp   vigorous_brown
166358d3b3ed   gcr.io/inverting-proxy/agent                       "/bin/sh -c '/opt/bi…"   2 weeks ago   Up 2 weeks                          proxy-agent


There are no running containers. **Note. If you are running this in at AI Notebook, you'll see one container running.**

The `hello-world` containers you ran previously already exited. In order to see all containers, including ones that have finished executing, run docker `ps -a`:

In [65]:
!docker ps -a

CONTAINER ID   IMAGE                                              COMMAND                  CREATED              STATUS                          PORTS                  NAMES
26591b2ae2b1   hello-world                                        "/hello"                 About a minute ago   Exited (0) About a minute ago                          festive_taussig
d5fea7d88c37   gcr.io/qwiklabs-gcp-04-569268d5f324/node-app:0.2   "node ./src/app.js"      2 weeks ago          Up 2 weeks                      0.0.0.0:4000->80/tcp   vigorous_brown
8ca7faf68b51   hello-world                                        "/hello"                 2 weeks ago          Exited (0) 2 weeks ago                                 competent_ishizaka
4f71022b9463   hello-world                                        "/hello"                 2 weeks ago          Exited (0) 2 weeks ago                                 zen_goodall
5d0057f91808   hello-world                                        "/hello"                 2 week

This shows you the Container ID, a UUID generated by Docker to identify the container, and more metadata about the run. The container Names are also randomly generated but can be specified with docker run --name [container-name] hello-world.

## Build a Docker container

Let's build a Docker image that's based on a simple node application.

**Exercise**

Open the text file called `intro.docker` in the `dockerfiles` folder and complete the TODO there. 

Your dockerfile should have the following steps

 1. use `FROM` to inherit an official Node runtime as the parent image; e.g. node:6
 2. use `WORKDIR` to seet the working directory to /app
 3. use `ADD` to copy the current directory to the container at /app
 4. use `EXPOSE` to make the containers port 80 available to the outside world
 5. use `CMD` to run the command `node ./src/app.js`

This file instructs the Docker daemon on how to build your image.

The initial line specifies the base parent image, which in this case is the official Docker image for node version 6.
In the second, we set the working (current) directory of the container.
In the third, we add the current directory's contents (indicated by the "." ) into the container.
Then we expose the container's port so it can accept connections on that port and finally run the node command to start the application.

Check out the other [Docker command references](https://docs.docker.com/engine/reference/builder/#known-issues-run) to understand what each line does.

We're going to use this Docker container to run a simple node.js app. Have a look at `app.js`. This is a simple HTTP server that listens on port 80 and returns "Hello World."


Now let's build the image. Note again the "`.`", which means current directory so you need to run this command from within the directory that has the Dockerfile.

The `-t` is to name and tag an image with the `name:tag` syntax. The name of the image is `node-app` and the tag is `0.1`. The tag is highly recommended when building Docker images. If you don't specify a tag, the tag will default to latest and it becomes more difficult to distinguish newer images from older ones. Also notice how each line in the Dockerfile above results in intermediate container layers as the image is built.

**Exercise**

Use `docker build` to build the docker image at `dockerfiles/intro.docker`. Tag the image `node-app:0.1`. 

In [79]:
!docker build -t gcr.io/${PROJECT_ID}/node-app/node-app:0.5 -f dockerfiles/intro.docker .

invalid argument "gcr.io/-gcp-04-569268d5f324/node-app/node-app:0.5" for "-t, --tag" flag: invalid reference format
See 'docker build --help'.


Let's check that the image has been created correctly. 

In [27]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED          SIZE
node-app                       0.1       8067ca1974b4   6 minutes ago    884MB
<none>                         <none>    895bef7b3195   8 minutes ago    884MB
<none>                         <none>    5b24b247208c   16 minutes ago   884MB
hello-world                    latest    d1165f221234   3 months ago     13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   3 months ago     1.73GB
node                           6         ab290b853066   2 years ago      884MB


You should see a `node-app` repository that was created only seconds ago. 

Notice `node` is the base image and `node-app` is the image you built. You can't remove `node` without removing `node-app` first. The size of the image is relatively small compared to VMs. Other versions of the node image such as `node:slim` and `node:alpine` can give you even smaller images for easier portability. The topic of slimming down container sizes is further explored in Advanced Topics. You can view all versions in the official repository here.

Note, you can remove an image from your docker images using `docker rmi [repository]:[tag]`.

## Run a Docker container

Now we'll run the container based on the image you built above using the `docker run` command. The `--name` flag allows you to name the container if you like. And `-p` instructs Docker to map the host's port 4000 to the container's port 80. This allows you to reach the server at http://localhost:4000. Without port mapping, you would not be able to reach the container at localhost.

In [28]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS                      PORTS     NAMES
8ca7faf68b51   hello-world                    "/hello"                 11 minutes ago   Exited (0) 11 minutes ago             competent_ishizaka
4f71022b9463   hello-world                    "/hello"                 18 minutes ago   Exited (0) 18 minutes ago             zen_goodall
5d0057f91808   hello-world                    "/hello"                 33 minutes ago   Exited (0) 33 minutes ago             cool_roentgen
166358d3b3ed   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   4 hours ago      Up 4 hours                            proxy-agent


**Exercise**

Use `docker run` to run the container you just build called `node-app:0.1`. Assign the host port `4000` to port `80` and assign it the name `my-app`.

In [29]:
!docker run -p 4000:80 --name my-app node-app:0.1

Server running at http://0.0.0.0:80/


To test out the server, open a terminal window and type the following command:

```bash
curl http://localhost:4000
```

You should see the server respond with `Hello World`

The container will run as long as the initial terminal is running. If you want to stop the container, run the following command in the terminal to stop and remove the container:

```bash
docker stop my-app && docker rm my-app
```
After a few moments the container will stop. You should notice the cell above will complete execution.

#### Running the container in the background
If you want to the container to run in the background (not tied to the terminal's session), you need to specify the `-d` flag.
Now run the following command to start the container in the background

**Exercise**

Modify your command above with `-d` flag to run `my-app` in the background.

In [30]:
!docker run -p 4000:80 --name my-app -d node-app:0.1

5e39fb4e45206af6a484ce5520a7fe96f33864641e815fa59a70a6a5f486b135


Your container is now running in the background. You can check the status of your running container using `docker ps`

In [31]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED         STATUS         PORTS                  NAMES
5e39fb4e4520   node-app:0.1                   "node ./src/app.js"      3 seconds ago   Up 2 seconds   0.0.0.0:4000->80/tcp   my-app
166358d3b3ed   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   4 hours ago     Up 4 hours                            proxy-agent


Notice the container is running in the output of docker ps. You can look at the logs by executing `docker logs [container_id]`. 

In [32]:
# Note, your container id will be different
!docker logs b9d5fd6b8e33

Error: No such container: b9d5fd6b8e33


You should see 
```bash
Server running at http://0.0.0.0:80/
```
If you want to follow the log's output as the container is running, use the `-f` option.

## Modify & Publish

Let's modify the application and push it to your Google Cloud Repository (gcr). After that you'll remove all local containers and images to simulate a fresh environment, and then pull and run your containers from gcr. This will demonstrate the portability of Docker containers.

### Edit `app.js`
Open the file `./src/app.js` with the text editor and replace "Hello World" with another string. Then build this new image. 

**Exercise**

After modifying the `app.js` file, use `docker build` to build a new container called `node-app:0.2` from the same docker file. 

In [67]:
!cat dockerfiles/intro.docker

# Use an official Node runtime as the parent image
FROM node:6

# Set the working directory in the container to /app
WORKDIR /app

# Copy the current directory contents into the container at /app
ADD . /app

# Make the container's port 80 available to the outside world
EXPOSE 80

# Run app.js using node when the container launches
CMD ["node", "./src/app.js"]

In [93]:
!docker build -t gcr.io/${PROJECT_ID}/node-app:0.4 -f dockerfiles/intro.docker .

invalid argument "gcr.io/-gcp-04-569268d5f324/node-app:0.4" for "-t, --tag" flag: invalid reference format
See 'docker build --help'.


Notice in `Step 2` of the output we are using an existing cache layer. From `Step 3` and on, the layers are modified because we made a change in `app.js`.

Run another container with the new image version. Notice how we map the host's port 8000 instead of 80. We can't use host port 4000 because it's already in use. 

**Exercise**

Run this new container in the background using a different port and with the name `my-app-2`.

In [69]:
!docker run -p 8000:80 --name my-app-2 -d node-app:0.3

0e274bbcf7b1f41c63e77b68c192dbedbc5976a3d304e5c8523412672ea510d3


You can check that both container are running using `docker ps`.

In [70]:
!docker ps

CONTAINER ID   IMAGE                                              COMMAND                  CREATED         STATUS         PORTS                  NAMES
0e274bbcf7b1   node-app:0.3                                       "node ./src/app.js"      3 seconds ago   Up 2 seconds   0.0.0.0:8000->80/tcp   my-app-2
d5fea7d88c37   gcr.io/qwiklabs-gcp-04-569268d5f324/node-app:0.2   "node ./src/app.js"      2 weeks ago     Up 2 weeks     0.0.0.0:4000->80/tcp   vigorous_brown
166358d3b3ed   gcr.io/inverting-proxy/agent                       "/bin/sh -c '/opt/bi…"   2 weeks ago     Up 2 weeks                            proxy-agent


And let's test boht containers using `curl` as before:

In [71]:
!curl http://localhost:8000

Hello Wolrd!


In [72]:
!curl http://localhost:4000

Hello Wolrd!


Recall, to stop a container running, you can execute the following command either in a terminal or (because they are running in the background) in a cell in this notebook. 

### Publish to gcr

Now you're going to push your image to the Google Container Registry (gcr). To push images to your private registry hosted by gcr, you need to tag the images with a registry name. The format is `[hostname]/[project-id]/[image]:[tag]`.

For gcr:

  * `[hostname]`= gcr.io
  * `[project-id]`= your project's ID
  * `[image]`= your image name
  * `[tag]`= any string tag of your choice. If unspecified, it defaults to "latest".

In [73]:
import os

PROJECT_ID = "qwiklabs-gcp-04-569268d5f324" # REPLACE WITH YOUR PROJECT NAME
os.environ["PROJECT_ID"] = PROJECT_ID

Let's tag `node-app:0.2`.

In [74]:
!docker images

REPOSITORY                                                              TAG       IMAGE ID       CREATED          SIZE
node-app                                                                0.3       eb62b5e5da54   35 seconds ago   884MB
gcr.io/qwiklabs-gcp-04-569268d5f324/tfx_covertype_continuous_training   latest    036086844ebc   12 days ago      4.98GB
gcr.io/qwiklabs-gcp-04-569268d5f324/tfx_covertype_continuous_training   <none>    9dd487f85433   12 days ago      4.98GB
gcr.io/qwiklabs-gcp-04-569268d5f324/node-app                            0.2       fe381b8df6e9   2 weeks ago      884MB
<none>                                                                  <none>    895bef7b3195   2 weeks ago      884MB
<none>                                                                  <none>    5b24b247208c   2 weeks ago      884MB
tensorflow/serving                                                      latest    e874bf5e4700   5 weeks ago      406MB
hello-world                            

**Exercise**

Tag the `node-app:0.2` image with a new image name conforming to the naming convention `gcr.io/[project-id]/[image]:[tag]`. Keep the image and tag names the same.

In [89]:
!cp dockerfiles/intro.docker ./Dockerfile

In [91]:
%%bash
gcloud build submit -t gcr.io/${PROJECT_ID}/node-app:0.4

ERROR: (gcloud) Invalid choice: 'build'.
Maybe you meant:
  gcloud builds submit

To search the help text of gcloud commands, run:
  gcloud help -- SEARCH_TERMS


CalledProcessError: Command 'b'gcloud build submit -t gcr.io/qwiklabs-gcp-04-569268d5f324/node-app:0.4\n'' returned non-zero exit status 2.

In [82]:
%%bash
docker tag node-app:0.3 gcr.io/${PROJECT_ID}/node-app:0.4

Now when we list our docker images we should see this newly tagged repository.

In [83]:
!docker images

REPOSITORY                                                              TAG       IMAGE ID       CREATED          SIZE
node-app                                                                0.4       727c458f19a6   12 minutes ago   884MB
node-app                                                                0.3       eb62b5e5da54   14 minutes ago   884MB
gcr.io/qwiklabs-gcp-04-569268d5f324/node-app                            0.3       eb62b5e5da54   14 minutes ago   884MB
gcr.io/qwiklabs-gcp-04-569268d5f324/node-app                            0.4       eb62b5e5da54   14 minutes ago   884MB
gcr.io/qwiklabs-gcp-04-569268d5f324/tfx_covertype_continuous_training   latest    036086844ebc   12 days ago      4.98GB
gcr.io/qwiklabs-gcp-04-569268d5f324/tfx_covertype_continuous_training   <none>    9dd487f85433   12 days ago      4.98GB
gcr.io/qwiklabs-gcp-04-569268d5f324/node-app                            0.2       fe381b8df6e9   2 weeks ago      884MB
<none>                                 

Next, let's push this image to gcr.

**Exercise**

Push this new image to the gcr.

In [80]:
%%bash
docker push gcr.io/${PROJECT_ID}/node-app:0.3

The push refers to repository [gcr.io/qwiklabs-gcp-04-569268d5f324/node-app]
530321e99aa0: Preparing
ab191a7b8846: Preparing
f39151891503: Preparing
f1965d3c206f: Preparing
a27518e43e49: Preparing
910d7fd9e23e: Preparing
4230ff7f2288: Preparing
2c719774c1e1: Preparing
ec62f19bb3aa: Preparing
f94641f1fe1f: Preparing
910d7fd9e23e: Waiting
4230ff7f2288: Waiting
2c719774c1e1: Waiting
ec62f19bb3aa: Waiting
f94641f1fe1f: Waiting
ab191a7b8846: Layer already exists
530321e99aa0: Layer already exists
f39151891503: Layer already exists
a27518e43e49: Layer already exists
f1965d3c206f: Layer already exists
910d7fd9e23e: Layer already exists
4230ff7f2288: Layer already exists
ec62f19bb3aa: Layer already exists
2c719774c1e1: Layer already exists
f94641f1fe1f: Layer already exists
0.3: digest: sha256:56e2e1415ea449258efb503f31811cc996e21ab5c7f5e2ea349af6c3dbde9fb5 size: 2424


Check that the image exists in `gcr` by visiting the image registry Cloud Console. You can navigate via the console to `Navigation menu > Container Registry` or visit the url from the cell below:

In [50]:
%%bash
echo "http://gcr.io/${PROJECT_ID}/node-app"

http://gcr.io/qwiklabs-gcp-04-569268d5f324/node-app


### Test the published gcr image

Let's test this image. You could start a new VM, ssh into that VM, and install gcloud. For simplicity, we'll just remove all containers and images to simulate a fresh environment.

First, stop and remove all containers using `docker stop` and `docker rm`. **Be careful not to stop the container running this AI Notebook!**.

In [51]:
!docker stop my-app && docker rm my-app

my-app
my-app


In [52]:
!docker stop my-app-2 && docker rm my-app-2

my-app-2
my-app-2


Now remove the docker images you've created above using `docker rmi`.

In [53]:
!docker images

REPOSITORY                                     TAG       IMAGE ID       CREATED          SIZE
gcr.io/qwiklabs-gcp-04-569268d5f324/node-app   0.2       fe381b8df6e9   5 minutes ago    884MB
node-app                                       0.2       fe381b8df6e9   5 minutes ago    884MB
node-app                                       0.1       8067ca1974b4   22 minutes ago   884MB
<none>                                         <none>    895bef7b3195   24 minutes ago   884MB
<none>                                         <none>    5b24b247208c   31 minutes ago   884MB
hello-world                                    latest    d1165f221234   3 months ago     13.3kB
gcr.io/inverting-proxy/agent                   <none>    fe507176d0e6   3 months ago     1.73GB
node                                           6         ab290b853066   2 years ago      884MB


In [57]:
%%bash
docker rmi node-app:0.2
docker rmi gcr.io/${PROJECT_ID}/node-app:0.2
docker rmi node-app:0.1
docker rmi node:6 
docker rmi -f hello-world:latest
docker rmi none:none

Error: No such image: none:none


CalledProcessError: Command 'b'docker rmi none:none\n'' returned non-zero exit status 1.

Confirm all images are removed with `docker images`.

In [58]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED          SIZE
<none>                         <none>    895bef7b3195   25 minutes ago   884MB
<none>                         <none>    5b24b247208c   32 minutes ago   884MB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   3 months ago     1.73GB


At this point you should have a pseudo-fresh environment. Now, pull the image and run it.

In [59]:
%%bash
docker pull gcr.io/${PROJECT_ID}/node-app:0.2
docker run -p 4000:80 -d gcr.io/${PROJECT_ID}/node-app:0.2

0.2: Pulling from qwiklabs-gcp-04-569268d5f324/node-app
c5e155d5a1d1: Already exists
221d80d00ae9: Already exists
4250b3117dca: Already exists
3b7ca19181b2: Already exists
425d7b2a5bcc: Already exists
69df12c70287: Already exists
ea2f5386a42d: Already exists
d421d2b3c5eb: Already exists
a159aaf8f7f4: Already exists
15440ee40a3d: Pulling fs layer
15440ee40a3d: Download complete
15440ee40a3d: Pull complete
Digest: sha256:424d8d840a58233141eff6f5fe8c2db25fd4a989e92ee16752deeba05c216801
Status: Downloaded newer image for gcr.io/qwiklabs-gcp-04-569268d5f324/node-app:0.2
gcr.io/qwiklabs-gcp-04-569268d5f324/node-app:0.2
d5fea7d88c376c7974743415417f91c55e6ca6877a143818f7cd88aaccc5e6b2


You can check that it's running as expected using before:

In [60]:
!curl http://localhost:4000

Hello Wolrd!


Copyright 2020 Google LLC Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.